In [ ]:
from pyscenic.rnkdb import RankingDatabase
from pyscenic.genesig import GeneSignature, Regulome
from pyscenic.recovery import enrichment, leading_edge
import matplotlib.pyplot as pyplot
from functools import partial
%matplotlib inline

In [2]:
GMT_FNAME = "/Users/u0043358/Projects/resources/c6.all.v6.1.symbols.gmt.txt"
DB_FNAME = "/Users/u0043358/Projects/databases/hg19-500bp-upstream-10species.mc9nr.db"

In [3]:
msigdb_c6 = GeneSignature.from_gmt(
                        fname=GMT_FNAME,
                        nomenclature="HGNC",
                        gene_separator="\t",
                        field_separator="\t")

In [4]:
msigdb_c6[0]

GeneSignature(name='GLI1_UP.V1_DN', nomenclature='HGNC', gene2weights=<frozendict {'COPZ1': 1.0, 'C10orf46': 1.0, 'C20orf118': 1.0, 'TMEM181': 1.0, 'CCNL2': 1.0, 'YIPF1': 1.0, 'GTDC1': 1.0, 'OPN3': 1.0, 'RSAD2': 1.0, 'SLC22A1': 1.0, 'LIN37': 1.0, 'RNF10': 1.0, 'RUNX1': 1.0, 'QRICH2': 1.0, 'SEPT9': 1.0, 'ARPC2': 1.0, 'CD2AP': 1.0, 'ACSL6': 1.0, 'ZMYND11': 1.0, 'BBX': 1.0, 'TMEM62': 1.0, 'FOLR1': 1.0, 'TMEM189': 1.0, 'PPIL1': 1.0, 'CYP3A7': 1.0, 'SELM': 1.0, 'ESYT1': 1.0, 'BTBD16': 1.0, 'SPECC1': 1.0}>)

In [5]:
db = RankingDatabase(fname=DB_FNAME,
                     name="hg19-500bp-upstream-10species",
                     nomenclature="HGNC")

In [6]:
df = enrichment(db, msigdb_c6[0]).sort_values(by=('Enrichment', 'NES'), ascending=False)

In [7]:
len(df)

24453

In [8]:
df.head()

Enrichment            \
                                                          AUC       NES   
tfdimers__MD00321                                    0.000184  4.653488   
transfac_pro__M05584                                 0.000182  4.583168   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...   0.000182  4.581865   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP       0.000179  4.488104   
transfac_pro__M05978                                 0.000177  4.424294   

                                                   Recovery                 \
                                                          0    1    2    3   
tfdimers__MD00321                                       0.0  0.0  0.0  0.0   
transfac_pro__M05584                                    0.0  0.0  0.0  1.0   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...      0.0  0.0  0.0  0.0   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP          0.0  0.0  0.0  0.0   
transfac_pro__M05978                                    0.0  0.0  0.0  0.0   

                                                                        \
                                                      4    5    6    7   
tfdimers__MD00321                                   0.0  0.0  0.0  0.0   
transfac_pro__M05584                                1.0  1.0  1.0  2.0   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...  0.0  0.0  0.0  0.0   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP      0.0  0.0  0.0  0.0   
transfac_pro__M05978                                0.0  0.0  0.0  0.0   

                                                     ...   Ranking         \
                                                     ...     RUNX1   SELM   
tfdimers__MD00321                                    ...       493  12664   
transfac_pro__M05584                                 ...        41  22270   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...   ...        10  22272   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP       ...       219   6533   
transfac_pro__M05978                                 ...      4713   3662   

                                                                         \
                                                   SEPT9 SLC22A1 SPECC1   
tfdimers__MD00321                                    245   14259  10114   
transfac_pro__M05584                                2944   12822      3   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...  2020   12996    432   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP       147   13590   4376   
transfac_pro__M05978                                 336     453  15219   

                                                                           \
                                                   TMEM181 TMEM189 TMEM62   
tfdimers__MD00321                                     6684   17288  16513   
transfac_pro__M05584                                   692    9615   7629   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...    9813    9816    190   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP        6854   13472  14829   
transfac_pro__M05978                                   393   14971  13768   

                                                                   
                                                    YIPF1 ZMYND11  
tfdimers__MD00321                                    2928   17277  
transfac_pro__M05584                                22079    9587  
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...  22086    9783  
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP      10499   15066  
transfac_pro__M05978                                  806     288  

[5 rows x 5031 columns]

In [9]:
len(df[df[('Enrichment', 'NES')] >= 2.5])

424

In [10]:
avgrcc = df['Recovery'].mean(axis=0)
stdrcc = df['Recovery'].std(axis=0)
avg2stdrcc = avgrcc + 2.0 * stdrcc

In [13]:
df[('Ennrichment', 'LE')] = df[df[('Enrichment', 'NES')] >= 2.5][['Recovery', 'Ranking']].apply(
    partial(leading_edge, avg2stdrcc=avg2stdrcc, genes=df['Ranking'].columns.values, nomenclature="HGNC"),
    axis=1)

In [12]:
df.head()

Enrichment            \
                                                          AUC       NES   
tfdimers__MD00321                                    0.000184  4.653488   
transfac_pro__M05584                                 0.000182  4.583168   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...   0.000182  4.581865   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP       0.000179  4.488104   
transfac_pro__M05978                                 0.000177  4.424294   

                                                   Recovery                 \
                                                          0    1    2    3   
tfdimers__MD00321                                       0.0  0.0  0.0  0.0   
transfac_pro__M05584                                    0.0  0.0  0.0  1.0   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...      0.0  0.0  0.0  0.0   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP          0.0  0.0  0.0  0.0   
transfac_pro__M05978                                    0.0  0.0  0.0  0.0   

                                                                        \
                                                      4    5    6    7   
tfdimers__MD00321                                   0.0  0.0  0.0  0.0   
transfac_pro__M05584                                1.0  1.0  1.0  2.0   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...  0.0  0.0  0.0  0.0   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP      0.0  0.0  0.0  0.0   
transfac_pro__M05978                                0.0  0.0  0.0  0.0   

                                                                          ...                          \
                                                                          ...                           
tfdimers__MD00321                                                         ...                           
transfac_pro__M05584                                                      ...                           
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...                        ...                           
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP                            ...                           
transfac_pro__M05978                                                      ...                           

                                                   Ranking                \
                                                      SELM SEPT9 SLC22A1   
tfdimers__MD00321                                    12664   245   14259   
transfac_pro__M05584                                 22270  2944   12822   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...   22272  2020   12996   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP        6533   147   13590   
transfac_pro__M05978                                  3662   336     453   

                                                                           \
                                                   SPECC1 TMEM181 TMEM189   
tfdimers__MD00321                                   10114    6684   17288   
transfac_pro__M05584                                    3     692    9615   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...    432    9813    9816   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP       4376    6854   13472   
transfac_pro__M05978                                15219     393   14971   

                                                                          \
                                                   TMEM62  YIPF1 ZMYND11   
tfdimers__MD00321                                   16513   2928   17277   
transfac_pro__M05584                                 7629  22079    9587   
taipale_tf_pairs__ETV2_ONECUT2_RCCGGAANNNNNATCG...    190  22086    9783   
taipale_tf_pairs__ETV5_FOXO1_RNCGGATGTTKWN_CAP      14829  10499   15066   
transfac_pro__M05978                                13768    806     288   

                                                                                          Ennrichment  
             